In [2]:
import pandas as pd
import numpy as np
import qiime2 as q2
import biom
from qiime2 import Artifact, Metadata, Visualization
from qiime2.plugins.emperor.actions import biplot
from qiime2.plugins.emperor.visualizers import plot
from qiime2.plugins.emperor.visualizers import biplot
from qiime2.plugins.qurro.actions import loading_plot
from qiime2.plugins.gemelli.actions import (joint_rpca, filter_ordination, feature_correlation_table)
from qiime2.plugins.gemelli.actions import rpca, qc_rarefy, phylogenetic_ctf_with_taxonomy, phylogenetic_rpca_with_taxonomy
from qiime2.plugins.longitudinal.actions import volatility
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.feature_table.methods import filter_samples
from biom import load_table

# from gemelli.rpca import rpca
from qiime2.plugins.gemelli.actions import rpca

# plotting
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import os

plt.style.use('ggplot')
%matplotlib inline

In [38]:
# load in biplot, biomtable, taxonomy, and metadata
table = Artifact.load("data/wolr2/gg2/zebra30/196551_zebra_2024_9.qza")
metadata = Metadata.load('data/metadata/15350_20240110-175840_noblanks.txt')
taxonomy = Artifact.load('data/wolr2/gg2/zebra30/196551_zebra_2024_9.taxonomy.qza')
metadata_df = metadata.to_dataframe()

In [39]:
metadata_df

,ados_cosotot,ados_cosototrrtot,ados_rrtot,ageatfecalsample,anonymized_name,antibiotics_last_2weeks,attend_daycare,breastfed_morethan1month,collection_date,collection_timestamp,...,sex,taxon_id,title,tube_id,vine_adapbehav_domstd,vine_comtotal_domstd,vine_dlytotal_domstd,vine_mtrtotal_domstd,vine_soctotal_domstd,zipcode
sample_name,,,,,,,,,,,,,,,,,,,,,
15350.A5U7C.2,8.0,14.0,6.0,35.0,X00235506,0.0,0,1.0,2023-02-08,2023-02-08,...,male,408170.0,Pierce ASD project,363146269.0,82.0,74.0,78.0,100.0,103.0,92026
15350.A6N5R.1,12.0,18.0,6.0,16.0,X00235660,0.0,0,1.0,2022-06-28,2022-06-28,...,male,408170.0,Pierce ASD project,363146304.0,95.0,96.0,95.0,102.0,99.0,92154
15350.A6T5B.1,1.0,2.0,1.0,14.0,X00235600,0.0,1,1.0,2022-04-18,2022-04-18,...,male,408170.0,Pierce ASD project,363146249.0,98.0,94.0,100.0,114.0,101.0,92102
15350.A7B7Z.1,20.0,27.0,7.0,24.0,X00235539,1.0,0,1.0,2022-06-21,2022-06-21,...,male,408170.0,Pierce ASD project,363146321.0,64.0,54.0,69.0,69.0,65.0,91950
15350.A7L6P.1,20.0,28.0,8.0,27.0,X00235658,0.0,0,1.0,2022-06-21,2022-06-21,...,male,408170.0,Pierce ASD project,363146256.0,74.0,60.0,92.0,89.0,75.0,91910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15350.C6Z3P.1,8.0,11.0,3.0,25.0,X00235753,0.0,0,1.0,2023-08-15,2023-08-15,...,female,408170.0,Pierce ASD project,363146274.0,76.0,54.0,92.0,89.0,86.0,91942
15350.C7T8B.1,9.0,14.0,5.0,34.0,X00235652,0.0,1,1.0,2023-10-13,2023-10-13,...,female,408170.0,Pierce ASD project,363146315.0,66.0,67.0,69.0,69.0,61.0,92111
15350.C7V5J.1,11.0,15.0,4.0,20.0,X00235548,0.0,0,1.0,2023-10-12,2023-10-12,...,male,408170.0,Pierce ASD project,363146252.0,94.0,92.0,95.0,95.0,101.0,92122


In [5]:
if not os.path.exists(f'data/wolr2/gg2/zebra30/rpca'):
    os.makedirs(f'data/wolr2/gg2/zebra30/rpca')

# ordination, distance = rpca(table, min_sample_count=500)
rpca_results = rpca(table, min_sample_count=0)

rpca_results.biplot.save("data/wolr2/gg2/zebra30/rpca/biplot.qza")
rpca_results.distance_matrix.save("data/wolr2/gg2/zebra30/rpca/distance_matrix.qza")
# rpca_results.ordination.save('data/wolr2/gg2/zebra30/rpca/ordination.qza')

biplot_viz = biplot(biplot=rpca_results.biplot, number_of_features=15, sample_metadata=metadata, feature_metadata=Metadata(taxonomy.view(pd.DataFrame)))
biplot_viz.visualization.save('data/wolr2/gg2/zebra30/rpca/biplot.qzv')

qurro_plot = loading_plot(rpca_results.biplot, table, metadata, feature_metadata=Metadata(taxonomy.view(pd.DataFrame)))
qurro_plot.visualization.save("data/wolr2/gg2/zebra30/rpca/qurro.qzv")

'data/wolr2/gg2/zebra30/rpca/qurro.qzv'

In [6]:
# all
significance_viz = beta_group_significance(
    distance_matrix=rpca_results.distance_matrix,
    metadata=metadata.get_column("dx"),
    method="permanova",
    pairwise=True
)

os.makedirs("data/wolr2/gg2/zebra30/rpca/permanova", exist_ok=True)
significance_viz.visualization.save("data/wolr2/gg2/zebra30/rpca/permanova/dx.qzv")

'data/wolr2/gg2/zebra30/rpca/permanova/dx.qzv'

<Figure size 640x480 with 0 Axes>

In [7]:
biom_table = table.view(biom.Table)
biom_table

304 x 83 <class 'biom.table.Table'> with 23410 nonzero entries (92% dense)

In [8]:
QC = True
if QC:
    from gemelli.rpca import rpca as gemelli_rpca
    from gemelli.utils import qc_rarefaction

    table = biom_table
    ordination, distance = gemelli_rpca(table, min_sample_count=10000)

    rare_ordination, rare_distance = gemelli_rpca(table.copy().subsample(10000))
    t, p = qc_rarefaction(table, rare_distance, distance)
    (t, p)

print(t, p)

-0.7657897072623497 0.4460549992165166


In [9]:
QC = True
if QC:
    from gemelli.rpca import rpca as gemelli_rpca
    from gemelli.utils import qc_rarefaction

    table = biom_table
    ordination, distance = gemelli_rpca(table, min_sample_count=0)

    rare_ordination, rare_distance = gemelli_rpca(table.copy().subsample(10000))
    t, p = qc_rarefaction(table, rare_distance, distance)
    (t, p)
print(t, p)


-0.5391682497611003 0.5912681561210946


# TP-TD

In [19]:
table = Artifact.load("data/wolr2/gg2/zebra30/196551_zebra_2024_9.qza")
metadata = Metadata.load('data/metadata/15350_20240110-175840_noblanks.txt')
taxonomy = Artifact.load('data/wolr2/gg2/zebra30/196551_zebra_2024_9.taxonomy.qza')
metadata_df = metadata.to_dataframe()


In [20]:

# Create dx_eyetrack column based on TP counts
tp_count = (metadata_df[['dx_compsocial', 'dx_geopref', 'dx_mtrtechno', 'dx_mtrtraffic', 'dx_outdoorplay']] == 'TP').sum(axis=1)

def get_dx_eyetrack(row, tp_count_value):
    if row['dx'] == 'TD':
        return -1
    elif tp_count_value == 0:
        return 0
    else:  # tp_count_value >= 1
        return 1

metadata_df['dx_eyetrack'] = metadata_df.apply(lambda row: get_dx_eyetrack(row, tp_count[row.name]), axis=1)

print(metadata_df['dx_eyetrack'].value_counts().sort_index())

# Filter to only TD (-1) or TP (1) samples
metadata_df_td_tp = metadata_df[metadata_df['dx_eyetrack'].isin([-1, 1])].copy()
print(metadata_df_td_tp['dx_eyetrack'].value_counts().sort_index())

-1    20
 0    34
 1    29
Name: dx_eyetrack, dtype: int64
-1    20
 1    29
Name: dx_eyetrack, dtype: int64


In [21]:
table_view = table.view(pd.DataFrame)

# Find samples that are in both the Table and metadata TD/TP
table_samples = set(table_view.index)
metadata_samples = set(metadata_df_td_tp.index)
common_samples = list(table_samples.intersection(metadata_samples))

print(f"\nSamples in table: {len(table_samples)}")
print(f"Samples in filtered metadata (TD + TP): {len(metadata_samples)}")
print(f"Common samples: {len(common_samples)}")

metadata_td_tp_filtered = metadata_df_td_tp.loc[common_samples]
metadata_td_tp_qiime = Metadata(metadata_td_tp_filtered)

table_filtered_result = filter_samples(
    table=table,
    metadata=metadata_td_tp_qiime
)

table_filtered = table_filtered_result.filtered_table


Samples in table: 83
Samples in filtered metadata (TD + TP): 49
Common samples: 49


In [23]:
# ordination, distance = rpca(table, min_sample_count=500)
rpca_results_tdtp = rpca(table_filtered, min_sample_count=0)

rpca_results_tdtp.biplot.save("data/wolr2/gg2/zebra30/rpca/biplot_tdtp.qza")
rpca_results_tdtp.distance_matrix.save("data/wolr2/gg2/zebra30/rpca/distance_matrix_tdtp.qza")


biplot_viz_tdtp = biplot(biplot=rpca_results_tdtp.biplot, number_of_features=15, sample_metadata=metadata_td_tp_qiime, feature_metadata=Metadata(taxonomy.view(pd.DataFrame)))
biplot_viz_tdtp.visualization.save('data/wolr2/gg2/zebra30/rpca/biplot_tdtp.qzv')

qurro_plot_tdtp = loading_plot(rpca_results_tdtp.biplot, table_filtered, metadata_td_tp_qiime, feature_metadata=Metadata(taxonomy.view(pd.DataFrame)))
qurro_plot_tdtp.visualization.save("data/wolr2/gg2/zebra30/rpca/qurro_tdtp.qzv")

'data/wolr2/gg2/zebra30/rpca/qurro_tdtp.qzv'

In [24]:
# all
significance_viz_tdtp = beta_group_significance(
    distance_matrix=rpca_results_tdtp.distance_matrix,
    metadata=metadata_td_tp_qiime.get_column("dx"),
    method="permanova",
    pairwise=True
)

significance_viz_tdtp.visualization.save("data/wolr2/gg2/zebra30/rpca/permanova/dx_tdtp.qzv")

'data/wolr2/gg2/zebra30/rpca/permanova/dx_tdtp.qzv'

<Figure size 640x480 with 0 Axes>

In [25]:
biom_table = table_filtered.view(biom.Table)
biom_table

304 x 49 <class 'biom.table.Table'> with 13807 nonzero entries (92% dense)

In [26]:
QC = True
if QC:
    from gemelli.rpca import rpca as gemelli_rpca
    from gemelli.utils import qc_rarefaction

    table = biom_table
    ordination, distance = gemelli_rpca(table, min_sample_count=10000)

    rare_ordination, rare_distance = gemelli_rpca(table.copy().subsample(10000))
    t, p = qc_rarefaction(table, rare_distance, distance)
    (t, p)

print(t, p)

-0.28481325350224684 0.7770655801212751


In [27]:
QC = True
if QC:
    from gemelli.rpca import rpca as gemelli_rpca
    from gemelli.utils import qc_rarefaction

    table = biom_table
    ordination, distance = gemelli_rpca(table, min_sample_count=0)

    rare_ordination, rare_distance = gemelli_rpca(table.copy().subsample(10000))
    t, p = qc_rarefaction(table, rare_distance, distance)
    (t, p)
print(t, p)


-0.5340547959152193 0.5958752904708084


# Phylo-RPCA

In [3]:
table = Artifact.load("data/wolr2/gg2/zebra30/196551_zebra_2024_9.qza")
metadata = Metadata.load('data/metadata/15350_20240110-175840_noblanks.txt')
taxonomy = Artifact.load('data/wolr2/gg2/zebra30/196551_zebra_2024_9.taxonomy.qza')
metadata_df = metadata.to_dataframe()
phylogeny = Artifact.import_data("Phylogeny[Rooted]", "ref/gg2/2024.09.phylogeny.asv.nwk")


In [4]:
phylo_rpca_results = phylogenetic_rpca_with_taxonomy(table, phylogeny, taxonomy.view(Metadata), min_sample_count=10000)
phylo_rpca_taxonomy = phylo_rpca_results.t2t_taxonomy.view(Metadata).to_dataframe()
phylo_rpca_results.biplot.save("data/wolr2/gg2/zebra30/rpca/phylo_biplot.qza")
phylo_rpca_results.distance_matrix.save("data/wolr2/gg2/zebra30/rpca/phylo_distance_matrix.qza")

'data/wolr2/gg2/zebra30/rpca/phylo_distance_matrix.qza'

In [10]:
from qiime2.plugins.empress.actions import community_plot

joint_tree_biplot = community_plot(phylo_rpca_results.counts_by_node_tree,
                                   phylo_rpca_results.counts_by_node,
                                   metadata,
                                   pcoa = phylo_rpca_results.biplot,
                                   feature_metadata = Metadata(phylo_rpca_taxonomy),
                                   number_of_features=50,
                                   ignore_missing_samples=False,
                                   filter_extra_samples=False,
                                   filter_missing_features=True)
joint_tree_biplot.visualization.save('data/wolr2/gg2/zebra30/rpca/phylo_rpca_joint_biplot.qzv')

/opt/homebrew/Caskroom/miniconda/base/envs/qiime2-shotgun-2024.2/lib/python3.8/site-packages/empress/tools.py:251: DataMatchingWarning: 303 feature(s) in the table were not present as tips in the tree. These feature(s) have been removed from the visualization.
  warnings.warn(


'data/wolr2/gg2/zebra30/rpca/phylo_rpca_joint_biplot.qzv'

In [11]:
qurro_plot = loading_plot(phylo_rpca_results.biplot, phylo_rpca_results.counts_by_node, metadata, feature_metadata = Metadata(phylo_rpca_taxonomy))
qurro_plot.visualization.save("data/wolr2/gg2/zebra30/rpca/phylo_rpca_qurro.qzv")

'data/wolr2/gg2/zebra30/rpca/phylo_rpca_qurro.qzv'